# PhysioMTL

The code below attempts to reproduce work from the 2022 paper “PhysioMTL: Personalizing Physiological Patterns using Optimal Transport Multi-Task Regression” (Zhu et al. 2022) [1]. The paper outlines Physiological Multitask-Learning, or PhysioMTL, a proposed method for improving heart rate variability (HRV) interpretation developed by resources at Apple Machine Learning Resource, Carnegie Mellon University, University of Illinois Urbana-Champaign, and University of Michigan. HRV is used to measure cardiovascular health and detect acute illnesses, but is too sensitive to irrelevant factors such as hydration, sleep, stress, etc. (Acharya et al. 2006; European Society of Cardiology and Pacing Electrophysiology 1996) [2]. PhysioMTL attempts to denoise and normalize HRV data. Specifically, it learns to predict heart rate patterns at specific times of day based on demographics (age, gender, activity level, stress, etc).

What makes PhysioMTL interesting is that it utilizes multi-task learning (MTL), treating patients as tasks. It then uses Optimal Transport (OT) methods to efficiently match data to unseen patients and adjust for data gaps. Overall, this is an early use of multi-task learning in healthcare.

The specific claims of the original paper we are testing in this code are as follows:

* PhysioMTL trained on a large sample set (80% of MMASH) outperforms (RMSE) the 5 baseline methods above
* PhysioMTL trained on a very small data set (20% of MMASH) outperforms (RMSE) the 5 baseline methods above
* PhysioMTL holds a more stable (lesser range) accuracy than the 5 baseline methods above across many data set training size (20%, 40%, 60%, and 80% of MMASH)

Each claim will also be evaluated against the 5 baseline methods of Group Lasso, Multi-level Lasso, Dirty models, Multi-task Wasserstein, and Reweighted Multi-task Wasserstein.

The following code will automatically download, process, train, and test the MMASH dataset and report results that support the claims presented in the paper. The notebook will generate an HRV prediction for a person who is 40 with a BMI of 25, sleeps 7 hours per day, has an activity level of 1.5 hours per day, and a stress level of 40. This prediction is also modeled in the original paper.

# Requirements & Imports

This code is intended to be ran on `Python 3.9`.

Multi-task regression library MuTaR is utilized to compare PhysioMTL evaluation with various other methods, including the following:

* Group Lasso
* Multi-Level Lasso
* Dirty Model
* MT Wasserstein
* Reweighted MT Wasserstein
* PhysioMTL

Python Optimal Transport (POT) and HWCounter are used for reporting of computational requirements and comparison of efficiency among the compared methods. These external libraries, along with other standard Python libraries, are mentioned below with version requirements:

* Numpy (1.22.4)
* Pandas (1.5.3)
* Matplotlib (3.7.1)
* Pot (0.9.0)
* Scipy (1.10.1)

In [1]:
import pandas
print(pandas.__version__)
import numpy
print(numpy.__version__)
import matplotlib
print(matplotlib.__version__)
import scipy
print(scipy.__version__)
# import pot
# print(pot.__version__)


1.5.3
1.26.4
3.7.1
1.11.4


In [2]:
!git clone https://github.com/hichamjanati/mutar
%cd mutar
!pip install -e .
%cd /content

# %pip install pot
%pip install hwcounter

import site
site.main()

Cloning into 'mutar'...
remote: Enumerating objects: 916, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 916 (delta 8), reused 16 (delta 7), pack-reused 899
Receiving objects: 100% (916/916), 1.72 MiB | 3.40 MiB/s, done.
Resolving deltas: 100% (495/495), done.
/content/mutar
Obtaining file:///content/mutar
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mutar
/content
  Preparing metadata (setup.py) ... done
  Created wheel for hwcounter: filename=hwcounter-0.1.1-cp310-cp310-linux_x86_64.whl size=11741 sha256=01dcf496834ba4e9e86e71e897dcc23a24dacb9a86e4172aa5e3a26d09be9556
  Stored in directory: /root/.cache/pip/wheels/1f/2e/b6/a4a88b10b8eb8ce2c79dceefa7302437ebff769e08bb673a90
Successfully built hwcounter


In [3]:

# # !pip install --force-reinstall numpy==1.23.4
# !pip install --force-reinstall pandas==1.5.3
# # !pip install --force-reinstall matplotlib==3.7.1
# # !pip install --force-reinstall scipy==1.10.1
# !pip install pot==0.9.0







In [4]:
# Evaluation of computational requirements, hardware logging, etc.
import platform
import socket
import re
import uuid
import json
import psutil
import logging

# Regression methods
import zipfile




import numpy as np



import pandas as pd
import ot
import matplotlib.cm as cm
import random
import mutar
from mutar import GroupLasso, DirtyModel, MTW, MultiLevelLasso, ReMTW
from hwcounter import Timer, count, count_end
from scipy import stats
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from sklearn.utils import shuffle

## Identify System Resources

These method definitions are used in various downstream classes to calculate precise computational requirements of both PhysioMTL and comparable methods. System information is printed and should be evaluated when comparing your results with results reported in our paper, as varation in CPU/memory and runtime can introduce considerable differences in CPU utilization. System requirements used in the creation of our paper are as follows:

* Architecture: x86_64
* RAM: 13GB
* CPU Cores (Physical): 1
* CPU Cores (Logical): 2
* Platform: Linux



In [5]:
def get_size(bytes):
    bucket = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < bucket:
            return f"{bytes:.2f}{unit}B"
        bytes /= bucket

def get_memory_utilization():
    svmem = psutil.virtual_memory()
    return {
        'Memory (Total)': f'{get_size(svmem.total)}',
        'Memory (Available)': f'{get_size(svmem.available)}',
        'Memory (Used)': f'{get_size(svmem.used)}',
        'Memory Utilization': f'{svmem.percent}%'
    }

def get_system_info():
    svmem = psutil.virtual_memory()
    return {
        'Platform': platform.system(),
        'Architecture': platform.machine(),
        'Processor': platform.processor(),
        'RAM': f'{round(psutil.virtual_memory().total / (1024 ** 3), 3)} GB',
        'CPU Cores (Physical)': psutil.cpu_count(logical=False),
        'CPU Cores (Total)': psutil.cpu_count(logical=True)
    } | get_memory_utilization()

def get_cpu_seconds(cycles):
    return cycles / psutil.cpu_count(logical=True) / 1000000000

print(get_system_info())

{'Platform': 'Linux', 'Architecture': 'x86_64', 'Processor': 'x86_64', 'RAM': '12.675 GB', 'CPU Cores (Physical)': 1, 'CPU Cores (Total)': 2, 'Memory (Total)': '12.67GB', 'Memory (Available)': '10.96GB', 'Memory (Used)': '1.40GB', 'Memory Utilization': '13.5%'}


# MMASH Data Set

The MMASH dataset contains 24-hour psycho-physiological data from 22 healthy adults. It includes data on respiratory rate, sleep quality, physical activity, anxiety, stress, and emotions Multilevel Monitoring of Activity and Sleep in Healthy People (MMASH) [3]

##Download MMASH

In [6]:
!mkdir MMASH
!mkdir Data
!mkdir Data/figures
!wget -O MMASH/mmash.zip -r -N -c -np https://physionet.org/files/mmash/1.0.0/MMASH.zip

with zipfile.ZipFile('MMASH/mmash.zip', 'r') as zip_ref:
  zip_ref.extractall('MMASH')

!rm MMASH/mmash.zip

will be placed in the single file you specified.

for details.

--2024-06-26 13:06:15--  https://physionet.org/files/mmash/1.0.0/MMASH.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24649620 (24M) [application/zip]
Saving to: ‘MMASH/mmash.zip’

MMASH/mmash.zip     100%[===================>]  23.51M   547KB/s    in 44s     

2024-06-26 13:07:00 (546 KB/s) - ‘MMASH/mmash.zip’ saved [24649620/24649620]

FINISHED --2024-06-26 13:07:00--
Total wall clock time: 45s
Downloaded: 1 files, 24M in 44s (546 KB/s)


##Preprocess MMASH

Preprocessing of the MMASH dataset is by converting respiratory rate to HRV in 5 minute intervals, omission of instances with a z-score over a threshold (2.5 by default). We also perform additional data cleansing to match behavior followed by the authors of the original paper, which included ignoring patient 4 (due to extreme HRV measurements) and populating missing sleep and age data on two other patients.

In [7]:
# Scripting constants
BASELINE_DATES = {1: "2020-08-01", 2: "2020-08-02"}
RANDOM_STATE = 12345
USER_FILE_PREFIX = "/content/MMASH/DataPaper/user_"
TASKS = range(1, 23)

# Data set variables
RELEVANT_ACTIVITY = [4, 5]
USER_INFO_FIELDS = ["Age", "Height", "Weight"]
OMIT_TASKS = [4,]
OVERWRITES = {3: [5,60], 11: [4,6.5], 18: [0,22]}
RAD_HOUR = 2.0 * np.pi / 24

# Pre-processing hyperparameters
IBIS_UPPER = 2.1
IBIS_LOWER = 0.3
FREQUENCY = "5min"
NOISE_UPPER = 55.0
NOISE_SAMPLE_PERCENT = 0.85

In [8]:
class MMASH:
    def __init__(self, tasks, max_zscore, max_noise=NOISE_UPPER):
        self.max_zscore = max_zscore
        if self.max_zscore is None:
            self.max_zscore = 999999
        self.max_noise = max_noise
        self.X, self.S, self.Y = [], [], []
        self.start_processing_cycles = count()

        for i in tasks:
            i_start = count()
            user_folder = USER_FILE_PREFIX + str(i)

            sleep_value = self.get_sleep_data(pd.read_csv(user_folder + "/sleep.csv"))
            activity_value = self.get_activity_data(pd.read_csv(user_folder + "/Activity.csv", header=0))
            user_info_value = self.get_user_info_data(pd.read_csv(user_folder + "/user_info.csv", header=0))
            questionnaire_value = self.get_questionnaire_data(pd.read_csv(user_folder + "/questionnaire.csv", header=0))
            rr_value = self.get_rr_data(pd.read_csv(user_folder + "/RR.csv", header=0))

            t_hour = rr_value["t_hour"].values
            s_vector = np.hstack([user_info_value, activity_value, sleep_value, questionnaire_value])
            if i in OVERWRITES:
                s_vector[OVERWRITES[i][0]] = OVERWRITES[i][1]

            self.X.append(np.asarray([np.sin(RAD_HOUR * t_hour), np.cos(RAD_HOUR * t_hour), np.ones(t_hour.shape[0], )]).T)
            self.S.append(s_vector.reshape(-1, 1))
            self.Y.append(rr_value["value"].values.reshape((-1, 1)))

            i_cycles = count_end() - i_start
            print(f'{i}: preprocessed task, y-mean = {rr_value["value"].mean()}, elapsed time: {i_cycles} cycles ({get_cpu_seconds(i_cycles)} seconds)')



        # reshaped_X = []

        # # Loop through each array in self.X
        # for x in self.X:
        #     # If the number of rows is greater than 176, truncate it
        #     if x.shape[0] > 176:
        #         reshaped_X.append(x[:176, :])
        #     # If the number of rows is less than 176, pad it
        #     else:
        #         # Calculate how many rows are needed to reach 176
        #         pad_size = 176 - x.shape[0]
        #         # Create a padding array of zeros with the appropriate shape
        #         pad_array = np.zeros((pad_size, x.shape[1]))
        #         # Concatenate the original array with the padding array
        #         reshaped_X.append(np.vstack((x, pad_array)))

        # # Optionally, assign back to self.X if you want to replace the original list
        # self.X = reshaped_X




        # reshaped_Y = []

        # # Loop through each array in self.Y
        # for y in self.Y:
        #     # If the number of rows is greater than 176, truncate it
        #     if y.shape[0] > 176:
        #         reshaped_Y.append(y[:176, :])
        #     # If the number of rows is less than 176, pad it
        #     else:
        #         # Calculate how many rows are needed to reach 176
        #         pad_size = 176 - y.shape[0]
        #         # Create a padding array of zeros with the appropriate shape
        #         pad_array = np.zeros((pad_size, y.shape[1]))
        #         # Concatenate the original array with the padding array
        #         reshaped_Y.append(np.vstack((y, pad_array)))

        # # Optionally, assign back to self.Y if you want to replace the original list
        # self.Y = reshaped_Y

        print([x.shape for x in self.X])
        print([x.shape for x in self.S])
        print([x.shape for x in self.Y])
        for i in range(len(self.X)):
            self.X[i]=np.array(self.X[i])
        for i in range(len(self.Y)):
            self.Y[i]=np.array(self.Y[i])
        for i in range(len(self.S)):
            self.S[i]=np.array(self.S[i])
        print(type(self.X),type(self.X[0]))


        # self.X = np.asarray(self.X)
        # self.S = np.asarray(self.S)
        # self.Y = np.asarray(self.Y)
        self.elapsed_processing_cycles = count_end() - self.start_processing_cycles
        print(f'finished processing mmash data set, elapsed cycles: {self.elapsed_processing_cycles} cycles ({get_cpu_seconds(self.elapsed_processing_cycles)} seconds)')

    def get_sleep_data(self, data):
        return data["Total Minutes in Bed"].sum() / 60

    def get_activity_data(self, data):
        data = data.copy().dropna()
        activity = lambda x: pd.to_timedelta(x["End"] + ":00") - pd.to_timedelta(x["Start"] + ":00")
        data["time_start_pd"] = data.apply(activity, axis=1)
        data["time_end_pd"] = data.apply(activity, axis=1)
        data["time_last_hour"] = data["time_end_pd"].apply(lambda x: x.seconds / 3600)
        data = data.loc[data["Activity"].isin(RELEVANT_ACTIVITY)]
        return data["time_last_hour"].sum() if not data.empty else 0

    def get_user_info_data(self, data):
        data = data[USER_INFO_FIELDS].loc[0].values
        return [data[0], data[1] / 100.0, data[2]]

    def get_questionnaire_data(self, data):
        return data["Daily_stress"].loc[0]

    def get_rr_data(self, data):
        data["new_time"] = data.apply(lambda ds: pd.to_datetime(BASELINE_DATES.get(ds["day"], BASELINE_DATES[2]) + " " + str(ds["time"])), axis=1)
        data['ibi_s'] = [x if x < IBIS_UPPER and x > IBIS_LOWER else np.nan for x in data['ibi_s']]
        data = [w for _, w in data.dropna().groupby(pd.Grouper(key='new_time', freq=FREQUENCY)) if not w.empty]
        rr_value = pd.DataFrame({
            "new_time": [x["new_time"].mean() for x in data],
            "value": [np.std(1000 * x["ibi_s"], ddof=1) for x in data]
        })
        no_noise = rr_value["value"] < self.max_noise
        no_noise = rr_value[no_noise].sample(int(NOISE_SAMPLE_PERCENT * no_noise.sum()), random_state=RANDOM_STATE)
        rr_value = rr_value.drop(no_noise.index)
        rr_value = rr_value.loc[np.abs(stats.zscore(rr_value["value"])) < self.max_zscore]
        rr_value = rr_value.loc[rr_value["value"] > 12]
        rr_value["t_hour"] = rr_value["new_time"].apply(lambda x: (x - pd.Timestamp(BASELINE_DATES[1] + " 00:00:00")).total_seconds() / 3600)
        return rr_value


In [9]:
tasks = [i for i in TASKS if i not in OMIT_TASKS]
mmash = MMASH(tasks, max_zscore=2.5)

1: preprocessed task, y-mean = 85.56476822678721, elapsed time: 24020697474 cycles (12.010348737 seconds)
2: preprocessed task, y-mean = 100.33561841324313, elapsed time: 16547829240 cycles (8.27391462 seconds)
3: preprocessed task, y-mean = 76.47079460368347, elapsed time: 22157179106 cycles (11.078589553 seconds)
5: preprocessed task, y-mean = 90.258288824083, elapsed time: 18260483648 cycles (9.130241824 seconds)
6: preprocessed task, y-mean = 88.1846116338921, elapsed time: 22268711848 cycles (11.134355924 seconds)
7: preprocessed task, y-mean = 85.14250546385614, elapsed time: 23031030042 cycles (11.515515021 seconds)
8: preprocessed task, y-mean = 96.14583988348869, elapsed time: 19757040754 cycles (9.878520377 seconds)
9: preprocessed task, y-mean = 88.87771062928078, elapsed time: 20004884304 cycles (10.002442152 seconds)
10: preprocessed task, y-mean = 96.38566520289122, elapsed time: 17597487180 cycles (8.79874359 seconds)
11: preprocessed task, y-mean = 87.05496543244831, el

# PhysioMTL Model Definition

The below PhysioMTL model is a multi-task learning model that treats every known patient as a task but does not follow a standard neural network pattern such as GNN or RNN. The algorithm learns a transformation matrix that maps the input features of each subject to a common feature space, and then learns a weight matrix that maps the common feature space to the output space of each subject.

The goal is to minimize the distance between predictions and true outputs across all subjects. The parameters are as follows:
* Input data (X) has 1 parameter per task for time.
* Source tasks (S) have 6 parameters for sleep, activity, stress, age, height, and weight.
* And target tasks (Y) have one parameter of HRV.

During the learning process for optimization/loss, the model first uses input data and target tasks to solve a weight matrix using linear regression. When it constructs cost and mapping matrices from source tasks, it uses weighted root mean squared error as a cost or objective function. Then it uses optimal transport (OT) to build a plan using the Sinkhorn algorithm, and a transformation matrix that maps demographic features to HRV. Using gradient descent, it alternates between optimizing the transformation matrix and OT plan until convergence. Once the transformation matrix is learned, the model can predict HRV data for new sets of demographic and time-of-day features.

In [10]:
T_GRADIENT_THRESHOLD = 1e-6
W_GRADIENT_THRESHOLD = 1e-6
COST_FUNC = lambda x, y: np.sqrt(np.mean(np.square(np.dot([1, 10., 1., 10, 1., 1.], x - y))))

In [17]:
import cvxpy as cp

from copy import copy


smallest_float = np.nextafter(np.float16(0), np.float16(1))
float_type = np.double


def compute_B(C, u, v, eta):

    return np.exp((u + v.T - C) / eta)

def sinkhorn_uot(C, a, b, eta=1.0, tau1=1.0, tau2=1.0, k=100, compute_optimal=False):
    """
    Sinkhorn algorithm for entropic-regularized Unbalanced Optimal Transport.

    :param C:
    :param a:
    :param b:
    :param eta:
    :param tau1:
    :param tau2:
    :param k:
    :param epsilon:
    :return:
    """

    output = {
        "u": list(),
        "v": list(),
        }



    # Initialization
    u = np.zeros_like(a).astype(float_type)
    v = np.zeros_like(b).astype(float_type)

    output["u"].append(copy(u))
    output["v"].append(copy(v))

    for i in range(int(k)):
        u_old = copy(u)
        v_old = copy(v)
        B = compute_B(C, u, v, eta)

        if i % 2 == 0:
            Ba = B.sum(axis=1).reshape(-1, 1)
            u = (u / eta + np.log(a) - np.log(Ba)) * (tau1 * eta / (eta + tau1))
        else:
            Bb = B.sum(axis=0).reshape(-1, 1)
            v = (v / eta + np.log(b) - np.log(Bb)) * (tau2 * eta / (eta + tau2))

        output["u"].append(copy(u))
        output["v"].append(copy(v))


    return output



In [12]:

data_dict_OT={
    "OT time":list(),
    "OT converge tim":list(),
    "PhysioMTL Mean":list(),
    "PhysioMTL OTS td":list(),
       }
data_dict_UOT={
    "UOT time":list(),
    "UOT converge tim":list(),
    "PhysioMTL UOT Mean":list(),
    "PhysioMTL UOT Std":list(),
       }
data_dict_other={
       "Percentage": list(),
       "Group Lasso Mean": list(), "Group Lasso Std": list(),
       "MLevel Lasso Mean": list(),"MLevel Lasso Std": list(),
       "Dirty Model Mean": list(), "Dirty Model Std": list(),
       "MT Was Mean": list(), "MT Was Std": list(),
       "MT Was Rewgt Mean": list(), "MT Was Rewgt Std":list()

       }
output={
        'parameters_OT':{ "converge_iterations_OT":list(),
                          "momentum":list()
                        },
        'parameters_UOT':{ "converge_iterations_UOT":list(),
                           "momentum":list(),
                           "eta":list(),
                           "tau":list(),
                           "k":list()
                        },
        'OT_result':data_dict_OT,
        'UOT_result':data_dict_UOT,
        }





In [26]:
import time
class PhysioMTL:
    def __init__(self, cost_func, iterations=50, t_learn_rate=0.055, t_iterations=200, w_learn_rate=5*1e-9, w_iterations=50, alpha=0.1, epsilon=1e-4, lambda_=5, sigma=10, verbose=False):
        self.cost_func = cost_func
        self.iterations = iterations
        self.t_learn_rate = t_learn_rate
        self.t_iterations = t_iterations
        self.w_learn_rate = w_learn_rate
        self.w_iterations = w_iterations
        self.alpha = alpha
        self.epsilon = epsilon
        self.lambda_ = lambda_   #tunable
        self.sigma = sigma
        self.verbose = verbose

    def fit(self, X, S, Y,Momentum=True):
        self.start_fit_cycles = count()
        self.S = S
        W = self.solve_w_by_linear_regression(X, Y)
        C, K = self.get_cost_matrix(S)
        Pi, count1 = self.solve_ot_plan(C)
        T=len(C)
        fracs = np.ones(T) / T
        self.coef_ = self.W, self.T = self.converge_fw_gradient_descent(X, Y, W, K, Pi,Momentum,UOT=False)
        self.end_fit_cycles = count_end() - self.start_fit_cycles
        if self.verbose:
          print(f'PhysioMTL fit completed in {self.end_fit_cycles} cycles ({get_cpu_seconds(self.end_fit_cycles)} seconds)')


    def fit_uot(self, X, S, Y,Momentum=True):
        self.start_fit_cycles = count()
        self.S = S
        W = self.solve_w_by_linear_regression(X, Y)
        C, K = self.get_cost_matrix(S)
        Pi = self.solve_uot_plan(C, eps=0.1, tau=3,k=30) #tune tau
        self.coef_ = self.W, self.T = self.converge_fw_gradient_descent(X, Y, W, K, Pi,Momentum,UOT=True )
        self.end_fit_cycles = count_end() - self.start_fit_cycles
        if self.verbose:
          print(f'PhysioMTL fit completed in {self.end_fit_cycles} cycles ({get_cpu_seconds(self.end_fit_cycles)} seconds)')


    def solve_w_by_linear_regression(self, X, Y):
        return np.concatenate([Y[i].T @ x @ np.linalg.inv(x.T @ x) for i, x in enumerate(X)]).T

    def get_cost_matrix(self, S):
        C = np.asarray([[self.cost_func(i, j) for j in S] for i in self.S])
        K = np.exp(-C / (2 * self.sigma**2))
        return C, K

    def solve_ot_plan(self, C):
        T = len(C)
        fracs = np.ones(T) / T
        transport_matrix = np.exp(-self.lambda_ * C)
        transport_matrix /= transport_matrix.sum()
        u = np.zeros(T)

        count=0
        while np.max(np.abs(u - transport_matrix.sum(1))) > self.epsilon:
            u = transport_matrix.sum(1)
            transport_matrix *= fracs / u[:, None]
            transport_matrix *= fracs / transport_matrix.sum(0)[None, :]
            count+=1
        return transport_matrix, count


    def solve_uot_plan(self, C, eps=0.1, tau=1, k=200):

      T = len(C)
      fracs = np.ones(T) / T
      eta=0.7
      uot_output = sinkhorn_uot(C, fracs , fracs, eta= eta, tau1=tau, tau2=tau, k= k, compute_optimal=True)
      u = uot_output["u"][-1]
      v = uot_output["v"][-1]
      transport_matrix = compute_B(C, u, v, eta)
      return transport_matrix


    def converge_fw_gradient_descent(self, X, Y, W, K, Pi,Momentum,UOT):
        N = len(X)
        Z = np.zeros((W.shape[0], K.shape[0]))
        T = np.zeros((W.shape[0], K.shape[0]))
        norm_last = 1e6
        if UOT==True:
           count=450
        else:
           count=1500
        if Momentum ==True:

              norm_last = 1e6
              for t in range(count):
                grad = [Pi[i, j] * (W[:, i:i+1] - T @ K[:, j:j+1]) @ K[:, j:j+1].T for i in range(N) for j in range(N)]
                grad = Z - self.alpha * 2 * sum(grad)
                norm = np.sum(np.square(grad))
                if np.abs(norm_last - norm) < T_GRADIENT_THRESHOLD:
                    break
                norm_last = norm
                if t==0:
                    v_t=0.011*grad
                else:
                    v_t=0.95*v_t+0.011* grad

                T = T - v_t


            #print(iteration, ": T gradient F-norm:", norm)

              norm_last = 1e6
              for w in range(count):
                grad = np.zeros_like(W)
                for t in range(N):
                    grad[:, t:t+1] = ((W[:, t:t+1].T @ X[t].T - Y[t].T) @ (X[t]) - self.alpha * sum([(T @ K[:, tt:tt+1] - W[:, tt:tt + 1]).T for tt in range(N)])).T
                norm = np.sum(np.square(grad))
                if np.abs(norm_last - norm) < W_GRADIENT_THRESHOLD:
                    break
                norm_last = norm
                if w==0:
                    v_t=1e-3*5* grad
                else:
                    v_t=0.9*v_t+1e-3*5* grad

                W = W - v_t


            #print(iteration, ": W gradient F-norm:", norm)
        if Momentum==False:
              norm_last = 1e6
              for t in range(count):
                grad = [Pi[i, j] * (W[:, i:i+1] - T @ K[:, j:j+1]) @ K[:, j:j+1].T for i in range(N) for j in range(N)]
                grad = Z - self.alpha * 2 * sum(grad)
                norm = np.sum(np.square(grad))
                if np.abs(norm_last - norm) < T_GRADIENT_THRESHOLD:
                    break
                norm_last = norm


                T=T-self.t_learn_rate*grad
            #print(iteration, ": T gradient F-norm:", norm)

              norm_last = 1e6
              for w in range(count):
                grad = np.zeros_like(W)
                for t in range(N):
                    grad[:, t:t+1] = ((W[:, t:t+1].T @ X[t].T - Y[t].T) @ (X[t]) - self.alpha * sum([(T @ K[:, tt:tt+1] - W[:, tt:tt + 1]).T for tt in range(N)])).T
                norm = np.sum(np.square(grad))
                if np.abs(norm_last - norm) < W_GRADIENT_THRESHOLD:
                    break
                norm_last = norm



                W=W-self.w_learn_rate*grad
            #print(iteration, ": W gradient F-norm:", norm)


        return W, T

    def predict(self, X, S):
        self.start_predict_cycles = count()
        if X is None and S is None:
            return [x @ self.W[:, i:i+1] for i, x in enumerate(self.X)]

        _, K = self.get_cost_matrix(S)
        K = [K[:, i:i+1] for i in range(len(S))]
        self.end_predict_cycles = count_end() - self.start_predict_cycles
        if self.verbose:
            print(f'PhysioMTL predict completed in {self.end_predict_cycles} cycles ({get_cpu_seconds(self.end_predict_cycles)} seconds)')

        return [x @ self.T @ K[i] for i, x in enumerate(X)]

# Train the model

In [18]:
# model = PhysioMTL(COST_FUNC, verbose=True)
# model.fit(mmash.X, mmash.S, mmash.Y)
# import time

model_ot = PhysioMTL(COST_FUNC, verbose=True)
model_ot.fit(mmash.X, mmash.S, mmash.Y)

model_uot = PhysioMTL(COST_FUNC, verbose=True)
model_uot.fit_uot(mmash.X, mmash.S, mmash.Y)









PhysioMTL fit completed in 13894605620 cycles (6.94730281 seconds)
PhysioMTL fit completed in 5797469508 cycles (2.898734754 seconds)


# Sample PhysioMTL Prediction

Perform a sample prediction against the trained model, and plot the predicted HRV values.

In [ ]:
S_ = np.array([23., 1.80, 85., 1., 7., 20.]).reshape(-1, 1)

T_ = np.linspace(8, 36, 30)
X_ = np.asarray([np.sin(RAD_HOUR * T_), np.cos(RAD_HOUR * T_), np.ones(30, )]).T

_, ax = plt.subplots()

ticks = [[9, '9:00'], [12, '12:00'], [15, '15:00'], [18, '18:00'], [21, '21:00'], [24, '24:00'], [27, '3:00'], [30, '6:00'], [33, '9:00'], [36, '12:00']]
ax.set_xticks([x[0] for x in ticks])
ax.set_xticklabels([x[1] for x in ticks], rotation=30)
ax.set_ylabel("HRV")
ax.set_xlabel("Time")

ax.plot(T_, model_uot.predict([X_], [S_])[0])
ax.plot(T_, model_ot.predict([X_], [S_])[0])
plt.show()

# Model Performance

The following evaluation methods are used to both assess performance of our model and address various claims made by the authors of the original PhysioMTL paper.  

## Evaluation Methods

In [19]:
def evaluate_all_percentages(X, S, Y, percentages, compare_alpha=0.9, physio_alpha=0.1, ot_gd=True, ot_gdm=True, uot_gd=True, uot_gdm=True , iterations=20, cost_func=COST_FUNC):
    for p in percentages:
        start_evaluation_time = count()
        evaluate_all(X, S, Y, p, compare_alpha, physio_alpha, iterations, cost_func,ot_gd=True, ot_gdm=True, uot_gd=True, uot_gdm=True,)
        end_evaluation_time = count_end() - start_evaluation_time
        print('    Evaluation:')
        print(f'         iterations: {iterations}')
        print(f'         elapsed time: {end_evaluation_time} cycles ({get_cpu_seconds(end_evaluation_time)} seconds)')
        print('')

def evaluate_all(X, S, Y, percentage, compare_alpha, physio_alpha, iterations, cost_func,ot_gd=True, ot_gdm=True, uot_gd=True, uot_gdm=True,):
    mse_ot_gd = []
    mse_glasso = []
    mse_mlasso = []
    mse_dirty = []
    mse_mtw = []
    mse_remtw = []
    mse_uot_gd= []
    mse_uot_gdm= []
    mse_ot_gdm = []
    time_ot_gd=[]
    time_ot_gdm=[]
    time_uot_gd=[]
    time_uot_gdm=[]


    for _ in range(iterations):
        cutoff = int(percentage * len(S))
        idx = list(range(0, len(S)))
        random.shuffle(idx)
        train_index, test_index = idx[:cutoff], idx[cutoff:]
        X_train = [X[i] for i in train_index]
        S_train = [S[i] for i in train_index]
        Y_train = [Y[i] for i in train_index]
        X_test = [X[i] for i in test_index]
        S_test = [S[i] for i in test_index]
        Y_test = [Y[i] for i in test_index]

        if ot_gd:
            model = PhysioMTL(cost_func, alpha=physio_alpha, verbose=False)
            time1=time.time()
            model.fit(X_train, S_train, Y_train, Momentum=False)
            time1=time.time()-time1
            time_ot_gd.append(time1)
            preds = model.predict(X_test, S_test)
            mse_ot_gd.append(get_rmse(preds, Y_test))

        if ot_gdm:
            model = PhysioMTL(cost_func, alpha=physio_alpha, verbose=False)
            time1=time.time()
            model.fit(X_train, S_train, Y_train, Momentum=True)
            time1=time.time()-time1
            time_ot_gdm.append(time1)
            preds = model.predict(X_test, S_test)
            mse_ot_gdm.append(get_rmse(preds, Y_test))

        if uot_gd:
            model = PhysioMTL(cost_func, alpha=physio_alpha, verbose=False)
            time1=time.time()
            model.fit_uot(X_train, S_train, Y_train,Momentum=False)
            time1=time.time()-time1
            time_uot_gd.append(time1)
            preds = model.predict(X_test, S_test)
            mse_uot_gd.append(get_rmse(preds, Y_test))
        if uot_gdm:
            model = PhysioMTL(cost_func, alpha=physio_alpha, verbose=False)
            time1=time.time()
            model.fit_uot(X_train, S_train, Y_train,Momentum=True)
            time1=time.time()-time1
            time_uot_gdm.append(time1)
            preds = model.predict(X_test, S_test)
            mse_uot_gdm.append(get_rmse(preds, Y_test))

        x = np.zeros((len(X_train), 100, 3 + 6))
        y = np.zeros((len(X_train), 100))
        for i, s in enumerate(S_train):
            idx = list(range(0, X_train[i].shape[0]))
            random.shuffle(idx)
            x[i:i+1, :, 0:3] = X_train[i][idx[:100]]
            y[i:i+1, :] = Y_train[i][idx[:100]].reshape((1, -1))
            x[i:i+1, :, 3:] = s.reshape((1, -1))

        x = np.zeros((len(X_train), 100, 3 + 6))
        y = np.zeros((len(X_train), 100))
        for i, s in enumerate(S_train):
            idx = list(range(0, X_train[i].shape[0]))
            random.shuffle(idx)
            x[i:i+1, :, 0:3] = X_train[i][idx[:100]]
            y[i:i+1, :] = Y_train[i][idx[:100]].reshape((1, -1))
            x[i:i+1, :, 3:] = s.reshape((1, -1))


    print(f'{percentage*100}% (alpha {compare_alpha} vs {physio_alpha})')

    if ot_gd:
        print(f'    PhysioMTL_GD:    {np.mean(mse_ot_gd)} ± {np.std(mse_ot_gd)}')
        print(f'    Time_GD:    {np.mean(time_ot_gd)} ± {np.std(time_ot_gd)}')

    if ot_gdm:
        print(f'    PhysioMTL_GDM:    {np.mean(mse_ot_gdm)} ± {np.std(mse_ot_gdm)}')
        print(f'    Time_GDM:    {np.mean(time_ot_gdm)} ± {np.std(time_ot_gdm)}')

    if uot_gd:
        print(f'    PhysioMTL_UOT_GD:    {np.mean(mse_uot_gd)} ± {np.std(mse_uot_gd)}')
        print(f'    Time_UOT_GD:    {np.mean(time_uot_gd)} ± {np.std(time_uot_gd)}')

    if uot_gdm:
        print(f'    PhysioMTL_UOT_GDM:    {np.mean(mse_uot_gdm)} ± {np.std(mse_uot_gdm)}')
        print(f'    Time_UOT_GDM:    {np.mean(time_uot_gdm)} ± {np.std(time_uot_gdm)}')


def get_rmse(a, b):
    return sum([np.sqrt(np.mean(np.square(a[i] - b[i]))) for i in range(len(a))]) / len(a)

def get_mse(model, X_train, Y_train, S_train, X_test, S_test, Y_test, cost_func):
    model.fit(X_train, Y_train)
    y = []
    for i, s_test in enumerate(S_test):
        idx = []
        for _, top_cost in sorted((cost_func(s, s_test), s) for s in S_train)[:1]:
            for j, s_train in enumerate(S_train):
                if np.max(s_train - top_cost) < 1e-6:
                    idx.append(j)
                    break

        X = np.zeros((len(X_test[i]), 9))
        X[:, 0:3] = X_test[i]
        X[:, 3:] = s_test.reshape((1, -1))
        y.append(X @ model.coef_[:, idx] + np.median(Y_train))

    return get_rmse(y, Y_test)

## Original Evaluation

Train the MMASH dataset using the author utilized alpha and Physio alpha values, among a variety of training ratios, using our PhysioMTL model as well as 5 other baseline models. Training across all models is done over 20 iterations.

In [27]:
# evaluate_all_percentages(mmash.X, mmash.S, mmash.Y, percentages=[0.2,0.4,0.6,0.8],physio_alpha=0.1,compare_alpha=None)

evaluate_all_percentages(mmash.X, mmash.S, mmash.Y, percentages=[0.2,0.4,0.6,0.8],physio_alpha=0.5,compare_alpha=None)

evaluate_all_percentages(mmash.X, mmash.S, mmash.Y, percentages=[0.2,0.4,0.6,0.8],physio_alpha=0.9,compare_alpha=None)

20.0% (alpha None vs 0.5)
    PhysioMTL_GD:    30.01207663418079 ± 0.6759605243053421
    Time_GD:    0.2924157381057739 ± 0.06678169738822465
    PhysioMTL_GDM:    30.11251241408663 ± 0.6860428654525352
    Time_GDM:    0.24371322393417358 ± 0.06655389325773069
    PhysioMTL_UOT_GD:    29.948401240796073 ± 0.7119027033995701
    Time_UOT_GD:    0.14179593324661255 ± 0.04337585257696374
    PhysioMTL_UOT_GDM:    30.03701248367297 ± 0.6721871136010001
    Time_UOT_GDM:    0.13691540956497192 ± 0.024345956624661727
    Evaluation:
         iterations: 20
         elapsed time: 36234755174 cycles (18.117377587 seconds)

40.0% (alpha None vs 0.5)
    PhysioMTL_GD:    30.11619006730868 ± 0.8508284343079405
    Time_GD:    1.4321748852729796 ± 0.4716538043397725
    PhysioMTL_GDM:    30.189257437451623 ± 0.8894879263934059
    Time_GDM:    0.8241727828979493 ± 0.3915318431934659
    PhysioMTL_UOT_GD:    30.034451797314272 ± 0.8735606706500648
    Time_UOT_GD:    0.5678562641143798 ± 0.138739

#Experiment & Ablation Evaluations

In each of the following sections, we attempt to recreate various claims made by authors of the original paper.

## Experiment 1: Validate Optimal Alpha

When the original authors trained the 5 baseline models for evaluation against PhysioMTL, they used a fixed alpha=0.9 (learning rate) for those models and alpha=0.1 for PhysioMTL. They stated that 0.9 was the optimal value for those other models.

The code within this section addresses this claim by evaluating all 5 models as well as PhysioMTL across 20 iterations, each with 20%, 40%, 60%, and 80% of the MMASH data, with 10 different alpha values (0.1 to 1.0). In another words, we evaluate all 5 models across 1,000 iterations using increasing amounts of sample data and learning rates.

### For Baseline Comparison Models

Perform experimental ablation for 5 baseline models (Group lasso, multi-level lasso, dirty model, multi-task wasserstein, and reweighted multi-taks wasserstein learning models). Note that alpha 0.9 is not evaluated here, as it was done so in the Original Evaluation section above.

## Experiment 2: Remove Denoising (Z-Score Max)

The original authors omitted HRV data more than 2.5 standard deviations from the mean, as part of denoising. Within this section, we assess this decision by training all models with 20%, 40%, 60%, and 80% of MMASH without this z-score denoising, to evaluate whether PhysioMTL still performs better.

In [ ]:
tasks = [i for i in TASKS if i not in OMIT_TASKS]
mmash_no_zmax = MMASH(tasks, max_zscore=None)
evaluate_all_percentages(mmash_no_zmax.X, mmash_no_zmax.S, mmash_no_zmax.Y, percentages=[0.2, 0.4, 0.6, 0.8])

## Experiment 3: Omit Unclean Sample Data

According to the original paper, the MMASH data set had some patients who were missing sleep and age data, so the authors coded assumed sleep and age numbers into the preprocessing. Doing this potentially risks confirming the authors’ biases that demographic data has very strong correlation to HRV, especially since MMASH is a relatively small data set. Thus, the code in this section performs an experiment in which we evaluate all models across 20%, 40%, 60%, and 80% of the MMASH data by excluding those patients, to see whether PhysioMTL still performs better.

In [ ]:
tasks = [i for i in TASKS if i not in OMIT_TASKS and i not in OVERWRITES]
mmash_no_unclean = MMASH(tasks, max_zscore=None)
evaluate_all_percentages(mmash_no_unclean.X, mmash_no_unclean.S, mmash_no_unclean.Y, percentages=[0.2, 0.4, 0.6, 0.8],compare_alpha=None)

# References

[1] Zhu, Jiacheng et al. (July 2022). “PhysioMTL: Personalizing Physiological Patterns using Optimal Transport Multi-Task Regression”. In: Proceedings of the Conference on Health, Inference, and Learning. Ed. by Gerardo Flores et al. Vol. 174. Proceedings of Machine Learning Research. PMLR, pp. 354–374. URL: https://proceedings.mlr.press/v174/zhu22a.html.

[2] Acharya, Rajendra U. et al. (2006). “Heart Rate Variability: A Review”. In: Medical and Biological Engineering and Computing 44.12, pp. 1031–1051.

[3] Rossi, Alessio et al. (2020). Multilevel Monitoring of Activity and Sleep in Healthy People (MMASH). version 1.0.0. URL: https://doi.org/10.13026/cerq-fc86.
